In [1]:
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from collections import Counter

In [2]:
torch.manual_seed(42)

In [3]:
class SVR(nn.Module):
    def __init__(self):
        super(SVR,self).__init__()
        self.fc1 = nn.Linear(4, 1)
        self.fc2 = nn.Linear(1, 1)
        
    def forward(self,x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

In [4]:
model=SVR()
# optimizer=torch.optim.SGD(model.parameters(),lr=0.0005)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

In [5]:
# loss加权重，减少数据标签数量差异的影响

In [6]:
wine_data = pd.read_csv('../data/epi_r_filtered_5.csv')
wine_data = wine_data.values
x = wine_data[:,1:]
x_s = preprocessing.StandardScaler().fit(x)
x = x_s.transform(x)
y = wine_data[:,0]

In [7]:
Counter(y)

Counter({5.0: 1732,
         2.5: 366,
         0.0: 1124,
         3.75: 3736,
         3.125: 1048,
         4.375: 5813,
         1.25: 108,
         1.875: 71})

In [8]:
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.3)

In [9]:
X = torch.from_numpy(x_train).type(torch.FloatTensor)
Y = torch.from_numpy(y_train).type(torch.FloatTensor)
X_t = torch.from_numpy(x_test).type(torch.FloatTensor)
Y_t = torch.from_numpy(y_test).type(torch.FloatTensor)

In [10]:
def calc_acc(predicted, real):
    precise_score = torch.round(predicted).detach().numpy()
    return accuracy_score(precise_score, real.detach().numpy())

In [11]:
TOTAL_EPOCH = 10000
for epoch in range(TOTAL_EPOCH):
    optimizer.zero_grad()
    predicted = model(X)
    loss = criterion(predicted, Y)
    loss.backward()
    optimizer.step()
    if epoch % 200 == 0:
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, TOTAL_EPOCH, loss))
#         print('train_score: {:.2f}'.format(calc_acc(predicted, Y)))
        with torch.no_grad():
            t_predicted = model(X_t)
#             print('test_score: {:.2f}'.format(calc_acc(t_predicted, Y_t)))
            print('test_loss: {:.4f}\n'.format(criterion(t_predicted, Y_t)))
        

/home/zfj/anaconda3/envs/torch1.4/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([9798])) that is different to the input size (torch.Size([9798, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch [1/10000], loss:19.5736
test_loss: 19.6945



/home/zfj/anaconda3/envs/torch1.4/lib/python3.7/site-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([4200])) that is different to the input size (torch.Size([4200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch [201/10000], loss:17.3465
test_loss: 17.4436

epoch [401/10000], loss:13.6004
test_loss: 13.6506

epoch [601/10000], loss:9.4891
test_loss: 9.5133

epoch [801/10000], loss:5.9603
test_loss: 5.9817

epoch [1001/10000], loss:3.5730
test_loss: 3.5729

epoch [1201/10000], loss:2.3111
test_loss: 2.2938

epoch [1401/10000], loss:1.8144
test_loss: 1.7860

epoch [1601/10000], loss:1.6731
test_loss: 1.6389

epoch [1801/10000], loss:1.6438
test_loss: 1.6072

epoch [2001/10000], loss:1.6404
test_loss: 1.6017

epoch [2201/10000], loss:1.6401
test_loss: 1.6007

epoch [2401/10000], loss:1.6405
test_loss: 1.6005



KeyboardInterrupt: 

In [ ]:
# 11-1 2400epoch loss：1.6405  test loss:1.6005